In [210]:
from sklearn.svm import SVR
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import string
import datetime
from datetime import timedelta
plt.style.use("fivethirtyeight")
import twint
from textblob import TextBlob
import base64
from io import BytesIO
import urllib
import plotly.express as px
# from plotly.offline.offline import _plot_html
import plotly.io.kaleido
import nest_asyncio
nest_asyncio.apply()
import PyQt5

In [211]:
def get_twints(date1,date2,q):
    c = twint.Config()
    c.Search = q
    # c.Username= "PMOIndia"
    # c.Verified=True
    c.Lang = "en"
    # c.Translate = True
    # c.TranslateDest = "hi"
  # Custom output format
    c.Since = date1
    c.Until = date2
    c.Limit = 1
    c.Pandas = True
    # c.Geo = "48.880048,2.385939,1km"
    # c.Database = "tweets.db"
    # c.Output="none.csv"
    return twint.run.Search(c)

In [212]:
def column_names():
    return twint.output.panda.Tweets_df.columns
def twint_to_pd(columns):
    return twint.output.panda.Tweets_df[columns]

def get_month(dt):
    s = dt[5:7]
    dt_obj = datetime.datetime.strptime(s,"%m")
    return dt_obj.strftime("%b")

In [213]:
def tweet_cleaner(text):
    text = re.sub('@[^\s]+','',text) #remove handlers
    text = re.sub('#\S+','',text) # remove hashtags
    text = re.sub(r'RT[\s]*','',text)
    text = re.sub("https*\S+",'',text) # remove urls
    text = re.sub(r':w*','',text)
    text = re.sub(r'\s+',' ',text,flags = re.I) # Substituting multiple spaces with single space
    text = re.sub(r'\w*\d+\w*', '', text) # remove numbers
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) # remove punctuations
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)


def get_sentiment(text):
    
    analysis = TextBlob(text)
    sentiment_polarity = analysis.sentiment.polarity
    sentiment_subjectivity = analysis.sentiment.subjectivity
    if sentiment_polarity > 0:
        sentiment_label = 'Positive'
    elif sentiment_polarity == 0:
        sentiment_label = 'Neutral'
    else: sentiment_label = 'Negative'
    result = {'polarity':sentiment_polarity, 
              'subjectivity':sentiment_subjectivity, 
              'sentiment':sentiment_label}
    return result


def set_time_line(df):
    timeline = df.groupby(['date', 'sentiment']).count().reset_index()
    timeline['count'] = timeline['tweet']
    timeline = timeline[['date', 'sentiment', 'count']]
  
    timeline = df.groupby(['date']).agg(np.nanmean).reset_index()
    timeline['count'] = df.groupby(['date']).count().reset_index()['nlikes']
    timeline = timeline[['date', 'count', 'polarity', 'nlikes','subjectivity']]
    timeline['polarity'] = timeline['polarity'].astype(float)
    timeline['subjectivity'] = timeline['subjectivity'].astype(float)
    return timeline

In [214]:
# def get_graph():
#     fig=plt.gcf()
#     buffer = BytesIO()
#     fig.savefig(buffer,format='png')
#     buffer.seek(0)
#     image_png = buffer.getvalue()
#     graph = base64.b64encode(image_png)
#     uri = 'data:image/png;base64,' + urllib.parse.quote(graph)
#     return uri
    
    
import plotly.graph_objects as go


def get_plot1(tweet,search):
    fi=px.histogram(tweet,x='sentiment',color='sentiment')
    fi.show()
    plt.switch_backend('AGG')
    plt.figure(figsize=(7,5))
    sns.set_theme(style="whitegrid")
    plt.figure(1)
    plt.title("Sentiment analysis of "+search)
    sns.countplot(tweet['sentiment'],palette="Set1")
    plt.xlabel('Sentiment Label')
    plt.ylabel('Tweet count')
    plt.xticks()
    plt.show()
    # plt.tight_layout()
    # graph=get_graph()
    return None

def get_plot2(sentiment,search):
    fi=px.histogram(sentiment,x='polarity')
    fi.update_xaxes(range=[-1,1])
    fi.update_traces(xbins=dict(start=-1,end=1,size=0.25))
    fi.update_layout(bargap=0.02)
    fi.show()
    plt.switch_backend('AGG')
    plt.figure(figsize=(7,5))
    sns.set_theme(style="whitegrid")
    plt.figure(1)
    plt.title("Sentiment From Tweets on "+search)
    sentiment.hist(bins=[-1, -0.75, -0.5, -0.25,0, 0.25, 0.5, 0.75, 1],color="darkred")
    plt.xlabel('Polarity Score')
    plt.ylabel('Tweet count')
    plt.xticks()
    plt.show()
    # plt.tight_layout()
    # graph=get_graph()
    return None

def get_plot3(t,search):
    # plt.figure(figsize=(10,8))
    fig1 = px.bar(t, x='date', y='count', color='polarity',title = search)
    fig2 = px.line(t, x="date", y="polarity", title='Polarity Vs Date')
    fig3 = px.bar(t, x='date', y='count', color='subjectivity',title = search)
    fig4 = px.line(t, x="date", y="subjectivity", title='Subjectivity Vs Date')
    fig2.update_yaxes(range=[-1,1])
    fig4.update_yaxes(range=[0,1])
    fig1.show()
    fig2.show()
    fig3.show()
    fig4.show()
    return None


# def get_graph2(fig):
#     image_png=fig.to_image(format='png')
#     graph = base64.b64encode(image_png).decode()
#     uri = 'data:image/png;base64,' + urllib.parse.quote(graph)
#     return uri

In [215]:
def find(start_date,end_date, search):
    list_of_dfs = []
    while(start_date!=end_date):
        date_obj = datetime.datetime.strptime(start_date,"%Y-%m-%d")
        delta = datetime.timedelta(days=1)
        date_obj +=delta
        start_date_2 = date_obj.strftime("%Y-%m-%d")
        get_twints(start_date,start_date_2,search)
        df=twint_to_pd(["tweet","date","nlikes"])
        start_date= start_date_2
        list_of_dfs.append(df)


    tweet = pd.concat(list_of_dfs)


    tweet['clean_tweets'] = tweet['tweet'].apply(tweet_cleaner)
    tweet['clean_tweets'] = tweet['clean_tweets'].apply(remove_emoji)
    tweet['sentiment_results'] = tweet['clean_tweets'].apply(get_sentiment)
    tweet = tweet.join(pd.json_normalize(tweet['sentiment_results']))
    tweet['sentiment'].value_counts()

    sentiment = pd.DataFrame(tweet[["polarity"]])
    sentiment = sentiment[sentiment.polarity != 0]

    tweet['date'] = pd.to_datetime(tweet['date']).dt.date
    t = set_time_line(tweet)  
    # print(tweet)
    return (get_plot1(tweet,search),get_plot2(sentiment,search),get_plot3(t,search))

In [216]:
print(find("2021-07-01","2021-07-02","narendra modi"))

1410749977467318281 2021-07-02 05:29:39 +0530 <AADIANADIANANT1> क्या यही है सेक्यूलर भारत  @PMOIndia @narendramodi @HMOIndia @AmitShahOffice @AmitShah
1410749970177613824 2021-07-02 05:29:38 +0530 <MANIBJP4PY> டெல்லி சென்ற புதுச்சேரி அமைச்சர் திரு AK.சாய் ஜெ சரவணன்குமார் அவர்கள் திரு G.கிஷன் ரெட்டி ஜி அவர்களை சந்தித்தார்., @AKSaiJSaravanan @kishanreddybjp @AmitShah @BlrNirmal @BJP4Puducherry @narendramodi @ApThiyagarajan @MANIBJP4PY  @KarnanM7 @HariJagadesh2  https://t.co/hrw2x1QsbT
1410749950036561929 2021-07-02 05:29:33 +0530 <mohanvenkat18> @sreeramjvc @MoHFW_INDIA @drharshvardhan @PMOIndia @narendramodi @COVIDNewsByMIB @PIB_India @CMOTamilnadu @TelanganaCMO @KTRTRS @IndiaAheadNews Sir,while if the allocation schedule to districts is available in Tamilnadu.If so in which domain.
1410749916524077056 2021-07-02 05:29:25 +0530 <ikrishnaacharya> आप कितनी भी कोशिश कर लीजिये @narendramodi जी इन लोगो का विश्वास जीतने की । इन जैसे लोग विश्वास लायक हैं ही नही ।  https://t.co/WRR463Ctr0
14107

C:\Users\agraw\AppData\Local\Programs\Python\Python39\lib\site-packages\seaborn\_decorators.py:36: FutureWarning:

Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.

C:\Users\agraw\AppData\Local\Temp/ipykernel_18936/1369091769.py:27: UserWarning:

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



C:\Users\agraw\AppData\Local\Temp/ipykernel_18936/1369091769.py:47: UserWarning:

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



(None, None, None)
